In [2]:
import pandas as pd
import pyensembl
import numpy as np
from types import MappingProxyType

In [3]:
#data.to_csv("~/projects/BIO/novel_junction.20230512.tsv",sep="\t",index=False)
data=pd.read_csv("~/projects/BIO/proteomics_MU_junction_all.20230613.tsv",sep="\t")

In [5]:
all_intron=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_all_introns.bed.gz",sep="\t",header=None,compression="gzip",usecols=[0,1,2,3,4,5,6,8],names=["chr","start","end","gene","gene_id","strand","transcript_id","annotation"])
protein_intron=all_intron[all_intron["annotation"]=="protein_coding"]

/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# in order to get the genes by individual.
def get_gene_list(genelist):
    data_gene_list=[]
    for value in genelist:
        value = value.split(',')
        data_gene_list.extend(value)
        
    return list(set(data_gene_list))

In [7]:
genome = pyensembl.Genome(
    reference_name='GRCh37',
    annotation_name='my_genome_lab',
    gtf_path_or_url='/home/ls/rachelcw/projects/protein_coding.gtf',
    transcript_fasta_paths_or_urls= '/private1/private/resources/Homo_sapiens_assembly19.fasta')
genome.index()

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /private1/private/resources/Homo_sapiens_assembly19.fasta.pickle


In [5]:
from Bio import SeqIO

# Specify the path to your FASTA file
fasta_file = "/home/ls/rachelcw/projects/BIO/cll_sf3b1_proteomics_novel1_protein.txt"

# Read the sequences from the FASTA file
seq_id=[]
sequences = []
for record in SeqIO.parse(fasta_file, "fasta"):
    sequence_id = record.id
    sequence = record.seq
    seq_id.append(sequence_id)
    sequences.append((sequence_id, sequence))

print(f"Found {len(sequences)} sequences")
# Print the sequences

# for sequence_id, sequence in sequences:
#     print(f"> {sequence_id}")
# print(sequence)

Found 969 sequences


*******************************************************
prepare data table- merge leafcutter ds results 

In [20]:
# mutated VS unmutated SF3B1 - group 2 - M+U CLL

# read in the data
cluster = pd.read_csv('~/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230512/fdr0.05/filtered.a2.20230512_cluster_significance.txt',sep=" ")
effect_size=pd.read_csv('~/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230512/fdr0.05/filtered.a2.20230512_effect_sizes.txt',sep=" ")

In [21]:
# filtered out junction mut(psi) > unmut(psi) --> delta psi < 0
mutated=effect_size[effect_size["deltapsi"]<0]

In [ ]:
mutated[["chr","start","end","cluster"]]=mutated["intron"].str.split(":",expand=True)
mutated["start"]=mutated["start"].astype(int)
mutated["end"]=mutated["end"].astype(int)
cluster[["chr","cluster"]]=cluster["cluster"].str.split(":",expand=True)
mutated=mutated.merge(cluster,on=["chr","cluster"],how="left")


In [23]:
data=mutated.filter(items=["intron","chr","start","end","genes","cluster","p.adjust"])
data.dropna(axis=0,inplace=True)

In [66]:
# gtf=pd.read_csv("/private/resources/gencode19_noChrPrefix_mitoMT.gtf",sep="\t", comment='#', header=None)
# all_intron=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_all_introns.bed.gz",sep="\t",header=None,compression="gzip")

In [7]:
# filter out intron that are not in the data
data_gene_list=get_gene_list(data['genes'].unique())
intron_in_data=protein_intron[protein_intron["gene"].isin(data_gene_list)]

In [26]:
data[['cluster','strand']]=data['cluster'].str.rsplit('_', expand=True, n=1)

In [29]:
# Duplicate rows based on comma-separated values
new_rows = []
for index, row in data.iterrows():
    values = row['genes'].split(',')
    if len(values) > 1:
        for i, value in enumerate(values):
            new_row = row.copy()
            new_row['genes'] = value
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Create updated DataFrame
data = pd.DataFrame(new_rows)


In [11]:
gene_dict={key: value for key, value in zip(protein_intron["gene"],protein_intron["gene_id"])}

In [14]:
len(protein_intron["gene"].unique())

18696

In [15]:
data["gene_id"]=data["genes"].map(gene_dict)

In [ ]:
# drop gene that arn't "protein coding" 
# data.dropna(axis=0,inplace=True)
data.isna().sum()

In [35]:
data["known"]=pd.NA
data["transcript_id"]=pd.NA
for gene in data_gene_list:
    # data_gene=data[data["genes"].str.contains(gene)]
    # intron_gene=intron_in_data[intron_in_data["gene"].str.contains(gene)]
    for index,row in data.iterrows():
        if ((intron_in_data["start"]==row["start"])&(intron_in_data["end"]==row["end"])).any():
            data.loc[index,"known"]="known"
            trans_id=intron_in_data.loc[(intron_in_data["start"]==row["start"])&(intron_in_data["end"]==row["end"]),"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        elif (intron_in_data["start"]==row["start"]).any():
            data.loc[index,"known"]="start"
            trans_id=intron_in_data.loc[intron_in_data["start"]==row["start"],"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        elif (intron_in_data["end"]==row["end"]).any():
            data.loc[index,"known"]="end"
            trans_id=intron_in_data.loc[intron_in_data["end"]==row["end"],"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        else:
            data.loc[index,"known"]="novel"
            transcript_of_intron=genome.transcript_ids_at_locus(contig=row.chr.replace("chr",""),position=row.end)
            if len(transcript_of_intron)>0:
                data.loc[index,"transcript_id"]=','.join(transcript_of_intron)      

In [20]:
data["gene_id"].isin(intron_in_data["gene_id"].unique()).sum()

1024

In [34]:
data.dropna(axis=0,inplace=True)

In [8]:
data.to_csv("~/projects/BIO/proteomics_MU_junction_all.20230613.tsv",sep="\t",index=False)

**********************************************************

In [15]:
# cassette exons #
known_intron=data[data["known"]=="known"]
known_intron["se"]=pd.NA
start_end_intron=[tuple(x) for x in intron_in_data[["start","end"]].values]
se=[tuple(x) for x in known_intron[["start","end"]].values if tuple(x) in start_end_intron]
# for index, row in known_intron.iterrows():
#     if row["start"].isin(intron_in_data["start"].unique()):
        
    

/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 24889859,  24890907],
       [ 71913725,  71914844],
       [ 71914880,  71917520],
       ...,
       [149098299, 149100807],
       [153736928, 153740181],
       [ 14954391,  14954989]])

In [6]:
# filter out intron that are known
novel_intron=data[data["known"]!="known"]

In [7]:
transcript_list=get_gene_list(novel_intron["transcript_id"])

len(transcript_list)

722

In [14]:
len(novel_intron["intron"].unique())

303

In [8]:
gene_list=get_gene_list(novel_intron["genes"].unique())

create dictionary: key-transcript id, values- exons-cds location of the transcript

In [ ]:
transcript_exons=dict()
for index, row in novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    for t in transcript:
        if t not in transcript_exons:
            exons=genome.exon_ids_of_transcript_id(t)
            locus=[genome.locus_of_exon_id(exon).to_tuple() for exon in exons]
            transcript_exons[t]=locus            
            
# create the dictionary constant
transcript_exons=MappingProxyType(transcript_exons)

In [7]:
counter_not_cds=0
counter_dup=0
transcript_strand_cds=dict()
for index, row in novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    chr=row["intron"].split(":")[0].replace("chr","")
    for t in transcript:
        if t not in transcript_strand_cds:
            try:
                cds=genome.transcript_by_id(t).coding_sequence_position_ranges
                transcript_strand_cds[t]=[chr,row["strand"],cds]
            except:
                # print(t)
                counter_not_cds+=1
                continue
        else:
            counter_dup+=1
           
print("counter_not_cds: ",counter_not_cds)
transcript_strand_cds=MappingProxyType(transcript_strand_cds)
print("counter_dup: ",counter_dup)

counter_not_cds:  39
counter_dup:  309


* known **start intron** = end exon1.
we want to find out the start of exon2, the closest exon to the end of the intron(start of the next exon) and replace the exon start with the novel intron end

* known **end intron** = start exon2.  
we want to find out the end of exon1,the closest exon to the start of the intron(end of the previous exon) and replace the exon end with the novel intron start

* known **novel intron**.
the intron is novel so we want to find the closest exons that the intron is between- the novel start exon2 and the novel end exon1

In [21]:
def get_novel_gene_locus_forward(transcript, start, end, known):
    locus=transcript_strand_cds[transcript][2].copy()
    start_cds=transcript_strand_cds[transcript][2][0][0]
    end_cds=transcript_strand_cds[transcript][2][-1][1]
    if known=="start":
        if end<start_cds:
            return None
        start_exon=np.array([l[0] for l in locus])
        index_exon=np.argmax(start_exon>end)
        if start==locus[index_exon][1]:
            index_exon=index_exon+1
        new_exon_locus=tuple([end,locus[index_exon][1]])
        locus[index_exon]=new_exon_locus
    elif known=="end":
        if start>end_cds:
            return None
        end_exon=np.array([l[1] for l in locus])
        index_exon=np.argmin(end_exon<start)
        if end==locus[index_exon][0]:
            index_exon=index_exon-1
        new_exon_locus=tuple([locus[index_exon][0],start])
        locus[index_exon]=new_exon_locus
    elif known=="novel":
        index_start=-1
        index_end=-1
        if end>start_cds:
            start_exon=np.array([l[0] for l in locus])
            index_start=np.argmax(start_exon>end)
            if start==locus[index_start][1]:
                index_start=index_start+1
        if start<end_cds:
            end_exon=np.array([l[1] for l in locus])
            index_end=np.argmin(end_exon<start)
            if end==locus[index_end][0]:
                index_end=index_end-1
        new_start=tuple([end,locus[index_start][1]])
        new_end=tuple([locus[index_end][0],start])
        if index_start!=-1:
            locus[index_start]=new_start
        if index_end!=-1:
            locus[index_end]=new_end
    return locus

In [15]:
transcript_gene_id=dict()
for index, row in novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    for t in transcript:
        transcript_gene_id[t]=row["gene_id"]

In [43]:
len(transcript_gene_id.keys())

722

In [47]:
# file known exons(cds) #
file_path="/home/ls/rachelcw/projects/BIO/cll_sf3b1_proteomics_original.txt"
rows=0# num of transcripts
with open(file_path,'w') as file:
    for t,value in transcript_strand_cds.items():
        chr=value[0]
        strand=value[1]
        cds=value[2]
        for loci in cds:
            file.write(f'{chr}\t{loci[0]}\t{loci[1]}\t{strand}\t{t}\t{transcript_gene_id[t]}\n')
        rows+=1
print(rows)

696


In [22]:
def get_novel_gene_locus_reverse(transcript, start, end, known):
    locus=transcript_strand_cds[transcript][2].copy()
    start_cds=transcript_strand_cds[transcript][2][0][1]
    end_cds=transcript_strand_cds[transcript][2][-1][0]
    if known == "start":
        if end>start_cds:
            return None
        start_exon=np.array([l[0] for l in locus])
        index_exon=np.argmin(start_exon>end)
        if start==locus[index_exon][1]:
            index_exon=index_exon-1
        new_exon_locus=tuple([end,locus[index_exon][1]])
        locus[index_exon]=new_exon_locus
        print(index_exon)
    if known == "end":
        if start<end_cds:
            return None
        end_exon=np.array([l[1] for l in locus])
        index_exon=np.argmax(end_exon<start)
        if end==locus[index_exon][0]:
            index_exon=index_exon+1
        new_exon_locus=tuple([locus[index_exon][0],start])
        locus[index_exon]=new_exon_locus
        print(index_exon)
    if known == "novel":
        index_start=-1
        index_end=-1
        if end<start_cds:
            start_exon=np.array([l[0] for l in locus])
            start_index=np.argmin(start_exon>end)
            if start==locus[start_index][1]:
                start_exon=start_index-1
            start_exon_locus=tuple([end,locus[start_index][1]])
        if start>end_cds:
            end_exon=np.array([l[1] for l in locus])
            end_index=np.argmax(end_exon<start)
            if end==locus[end_index][0]:
                end_index=end_index+1
        if index_start!=-1:
            new_start=tuple([end,locus[index_start][1]])
            locus[index_start]=new_start
        if index_end!=-1:
            new_end=tuple([locus[index_end][0],start])
            locus[index_end]=new_end
    return locus


In [23]:
# file novel exons(cds) #
file_path="/home/ls/rachelcw/projects/BIO/cll_sf3b1_proteomics_novel.txt"
rows=0 # num of transcripts
t_not_cds=0 # num of transcripts not in cds
events=0
with open(file_path,'w') as file:
    for index, row in novel_intron.iterrows():
        transcript=row["transcript_id"].split(",")
        events+=(len(transcript))
        for t in transcript:
            if t not in transcript_strand_cds.keys():
                t_not_cds+=1
                continue
            locus=[]
            rows+=1
            strand=transcript_strand_cds[t][1]
            if strand=="-":
                locus=get_novel_gene_locus_reverse(t,row["start"],row["end"],row["known"])
            else: # strand=="+"
                locus=get_novel_gene_locus_forward(t,row["start"],row["end"],row["known"])
            if locus is None:
                continue
            chr=transcript_strand_cds[t][0]
                
            for loci in locus:
                file.write(f'{chr}\t{loci[0]}\t{loci[1]}\t{strand}\t{t}\t{transcript_gene_id[t]}\t{row["intron"]}\n')
                # print(f'{chr}\t{loci[0]}\t{loci[1]}\t{strand}\t{t}\t{transcript_gene_id[t]}\t{row["intron"]}\n') #
print(rows)
print(t_not_cds)
print(events)
# t=="ENST00000376510.3" --> +
# t=="ENST00000378927.3" --> -
# t=="ENST00000412050.4" --> -

1
new [(12228228, 12228290), (12222394, 12226955), (12221082, 12221131), (12219792, 12219899), (12215717, 12215812), (12214764, 12214865), (12212855, 12212907)]
2
new [(12228228, 12228290), (12226888, 12226955), (12222394, 12226531), (12221082, 12221131), (12219792, 12219899), (12215717, 12215812), (12214764, 12214865), (12212899, 12212907), (12212716, 12212769), (12209704, 12209810)]
1
new [(12228228, 12228290), (12222394, 12226955), (12221082, 12221131), (12219792, 12219899), (12215717, 12215812), (12214764, 12214865), (12212899, 12212907), (12212716, 12212769), (12212148, 12212155)]
1
new [(12228228, 12228290), (12222394, 12226955), (12221028, 12221131), (12219842, 12219899)]
1
new [(12228228, 12228290), (12222394, 12226955), (12221082, 12221131), (12219792, 12219899), (12215717, 12215812), (12214764, 12214865), (12212899, 12212907), (12212716, 12212769), (12209704, 12209810)]
1
new [(12228228, 12228290), (12222394, 12226955), (12221082, 12221131), (12219792, 12219899), (12215717, 1

************************************************************************

In [26]:
import pyfaidx as fa
fasta =fa.Fasta("/private1/private/resources/Homo_sapiens_assembly19.fasta")
end=72463448-203
s=fasta["10"][5944983-1:5945138].seq
s1=fasta["10"][5944983:5945138].seq
print(s)
print(s1)
# print(len(s))
# print(s1[0:100])




TGAGACGGTTTAAGCGGAAGCATCTTACTGCCATTGACTGCCAGCATTTGGCTCGGAGTCACTTGGCTGTGACCCAGCCCTTCGGTCAAAGATGGACAAACAGAGATCCGAACCATGGTCTCTATCCTAAACCGAGAACAAAAAGAGGGAGTAGGG
GAGACGGTTTAAGCGGAAGCATCTTACTGCCATTGACTGCCAGCATTTGGCTCGGAGTCACTTGGCTGTGACCCAGCCCTTCGGTCAAAGATGGACAAACAGAGATCCGAACCATGGTCTCTATCCTAAACCGAGAACAAAAAGAGGGAGTAGGG


In [112]:
# genome.exon_ids_of_transcript_id("ENST00000368644.1")
# genome.coding_sequence("ENST00000368644.1")

genome.transcript_by_id("ENST00000393609.3").coding_sequence_position_ranges

[(72437665, 72438173),
 (72425197, 72425366),
 (72424221, 72424288),
 (72423483, 72423613),
 (72423241, 72423384),
 (72422475, 72422544),
 (72422066, 72422186),
 (72421430, 72421632),
 (72420915, 72421021),
 (72418220, 72418419),
 (72416850, 72416935),
 (72415197, 72415379),
 (72413950, 72414124),
 (72412694, 72412828),
 (72410462, 72410597),
 (72410050, 72410152),
 (72408956, 72409151),
 (72408643, 72408694),
 (72408368, 72408531),
 (72408028, 72408240),
 (72407594, 72407699),
 (72406763, 72406910),
 (72406587, 72406673),
 (72406432, 72406500),
 (72406025, 72406142),
 (72404737, 72404850),
 (72404370, 72404515),
 (72403798, 72403830),
 (72399500, 72399582),
 (72398733, 72398783),
 (72398484, 72398547),
 (72397087, 72397236),
 (72396712, 72396726)]

In [29]:
t=genome.transcript_sequence(transcript_id='ENST00000393605.3')

In [57]:
import collections
transcript=collections.namedtuple("transcript",['chr', 'start', 'end','strand','gene_id'])
transcript=transcript(11,1,2,'-','ENSG')
d={"intron1":[{"ENST":transcript}]}
d["intron1"].append({"ENST1":[transcript._make([11,1,2,'-','ENSG2'])]})
d["intron1"]["ENST1"].append(transcript._make([12,1,2,'-','ENSG2']))
d["intron2"]={"ENST2":transcript._make([11,1,2,'-','ENSG2'])}
print(d)
for i in d.keys():
    print(i)
    for j in d[i].keys():
        print(j)

TypeError: list indices must be integers or slices, not str

In [ ]:
# # option to frame shift- need to change the modulo to the len of the gene until the new locus # #
# start_novel_intron=novel_intron[novel_intron["novel"]=="start"]
# # start_novel_intron["frameshift"]=0
# for index, row in start_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     t_frame=[]
#     for t in transcript:
#         start_exon=np.array([l[1] for l in transcript_exons[t]])
#         print(row["end"])
#         print(start_exon)
#         index_exon=np.abs(start_exon-row["end"]).argmin()
#         print(index_exon)
#         print("old",transcript_exons[t][index_exon])
#         new_exon_locus=tuple([transcript_exons[t][index_exon][0],row["end"],transcript_exons[t][index_exon][2],transcript_exons[t][index_exon][3]])
#         transcript_exons_updated[t][index_exon]=new_exon_locus
#         if (transcript_exons[t][index_exon][2]-transcript_exons[t][index_exon][1]-1)%3!=0:
#             print("new",new_exon_locus)
#             t_frame.append(t)
#     start_novel_intron.loc[index,"frameshift"]=','.join(t_frame)
        
        


In [57]:
# # my tryint to create a dictionary of {intron(junction):{t_is:[exons locus]}} ##

# intron_transcript=dict()
# for index, row in novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     intron_transcript[row["intron"]] = [{t: transcript_exons[t]} for t in transcript]

# start_novel_intron=novel_intron[novel_intron["novel"]=="start"]
# # start_novel_intron["frameshift"]=0
# for index, row in start_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     for i,t in enumerate(transcript):
#         start_exon=np.array([l[1] for l in transcript_exons[t]])
#         index_exon=np.argmax(start_exon>row["end"])
#         if row["start"]==transcript_exons[t][index_exon][2]:
#             index_exon=index_exon+1
#         new_exon_locus=tuple([transcript_exons[t][index_exon][0],row["end"],transcript_exons[t][index_exon][2],transcript_exons[t][index_exon][3]])
#         # print(intron_transcript[row["intron"]][i][t][index_exon])
#         intron_transcript[row["intron"]][i][t][index_exon]=new_exon_locus
#         # print(intron_transcript[row["intron"]][i][t][index_exon])
#         # print(t)
#         # print(row["end"])
#         # print(start_exon)
#         # print(index_exon)
#         # print("old",transcript_exons[t][index_exon])
#         # print(transcript_exons[t])
#         # print("new",new_exon_locus)
           
# end_novel_intron=novel_intron[novel_intron["novel"]=="end"]
# # start_novel_intron["frameshift"]=0
# for index, row in end_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     # t_frame=[]
#     for i,t in enumerate(transcript):
#         end_exon=np.array([l[2] for l in transcript_exons[t]])
#         index_exon=np.argmin(end_exon<row["start"])
#         if row["end"]==transcript_exons[t][index_exon][1]:
#             index_exon=index_exon-1
#         new_exon_locus=tuple([transcript_exons[t][index_exon][0],transcript_exons[t][index_exon][1],row["start"],transcript_exons[t][index_exon][3]])
#         intron_transcript[row["intron"]][i][t][index_exon]=new_exon_locus
#         # print(end_exon<row["start"])
#         # print(t)
#         # print(row["start"])
#         # print(end_exon)
#         # print(index_exon)
#         # print("old",transcript_exons[t][index_exon])
#         # print(transcript_exons[t])
#         # print("new",new_exon_locus)

# novel_novel_intron=novel_intron[novel_intron["novel"]=="novel"]
# # start_novel_intron["frameshift"]=0
# for index, row in novel_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     # t_frame=[]
#     for i,t in enumerate(transcript):
#         start_exon=np.array([l[1] for l in transcript_exons[t]])
#         end_exon=np.array([l[2] for l in transcript_exons[t]])
#         index_start=np.argmax(start_exon>row["end"])
#         if row["start"]==transcript_exons[t][index_start][2]:
#             index_start=index_start+1
#         index_end=np.argmin(end_exon<row["start"])
#         if row["end"]==transcript_exons[t][index_end][1]:
#             index_end=index_end-1
#         new_start=tuple([transcript_exons[t][index_start][0],row["end"],transcript_exons[t][index_start][2],transcript_exons[t][index_start][3]])
#         new_end=tuple([transcript_exons[t][index_end][0],transcript_exons[t][index_end][1],row["start"],transcript_exons[t][index_end][3]])
#         intron_transcript[row["intron"]][i][t][index_start]=new_start
#         intron_transcript[row["intron"]][i][t][index_end]=new_end

In [ ]:
# pyensembl.search.find_nearest_locus(start, end, loci)
# Finds nearest locus (object with method distance_to_interval) to the interval defined by the given start and end positions. Returns the distance to that locus, along with the locus object itself.

In [49]:
non_protein=get_gene_list(novel_intron.loc[novel_intron["transcript_id"]=='',"genes"].unique())

In [60]:
len(novel_intron.iloc[191]["transcript_id"])

0

In [54]:
protein_intron.loc[protein_intron["gene"].isin(non_protein),"gene"].unique()

array(['PLGLB2'], dtype=object)

In [59]:
genome.transcript_ids_at_locus(contig="2",position=88037624)

[]

In [21]:
genome.transcript_ids_of_exon_id('ENSE00000728678.1')

ValueError: No results found for query:

            SELECT distinct transcript_id
            FROM transcript
            WHERE exon_id = ?
        
with parameters: ['ENSE00000728678.1']

In [62]:
genome.transcript_ids_at_locus(contig="10",position=131960844,end=131964771,strand="+")

['ENST00000331244.5', 'ENST00000368644.1', 'ENST00000481034.1']